In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import remote_control

remote_control.__file__

In [ ]:
from remote_control import acquisition

## calibration_positions is used to calculate the Z coordinate for each pixel
calibration_positions = [
         (422, 4252, -4.4), # Top-left
         (-10252, 4187, -94.5), # Top-right
         (29, 42684, 122.5), # Bottom-left
         (-10536, 42666, 43) # Bottom-right
]

## target_positions is used to optionally select the target area, if it's different from the calibration positions.
## This is optional and can just be set to calibration_positions if they're the same positions
target_positions = calibration_positions
# target_positions = [
#          (400, 2000),
#          (-10000, 2000),
#          (400, 20000),
#          (-10000, 20000)
# ]

ra = acquisition.RectangularAquisition(config_fn = 'remote_config.json', 
                                       datadir="./data", # will save spatial position file here
                                       log_fname="./test.log")

image_bounds = [
                  [3700, -450],
                  [-13900, 48000],
                ] # [.max_x, min_y] [min_x, .max_y] 
ra.set_image_bounds(image_bounds)

## Use x_pitch/y_pitch to generate the grid with a specific pitch:
ra.generate_targets(calibration_positions, target_positions, x_pitch=100, y_pitch=100)

## Use x_size/y_size to generate grid with a specific size:
# ra.generate_targets(calibration_positions, target_positions, x_size=200, y_size=500)

## Use interpolate_xy to allow trapezoidal grids
# ra.generate_targets(calibration_positions, target_positions, x_size=200, y_size=500, interpolate_xy=True)

## Use an image mask if there are areas to avoid
# ra.apply_image_mask('rect_acquisition_mask.png')

# Display a preview
ra.plot_targets()

In [ ]:
# Prefix for output files
filename = "./data/01052019_Mouse_DHB_pos_mz200-800_px50x50_LR"
ra.acquire(filename, dummy=True)